# 相关系数r，线性回归与方差分析

代码运行方式，进入一个代码块，按ctrl+enter运行代码

## 相关系数

线性相关系数的公式是

$r = \frac{cov(x,y)}{\sqrt{VAR(x)VAR(y)}}$
$=\frac{\sum{(x-\bar{x})(y-\bar{y})}}{\sqrt{\sum{(x-\bar{x})^2}\sum{(y-\bar{y})^2}}}$

下面的代码演示，当x和y分别相对于$\bar{x}$和$\bar{y}$同步变化的时候，r趋近于1，反向变化时趋近于-1，关联性差的时候趋近于0

In [ ]:
# library loading
library(dplyr)
library(ggplot2)
library(plotly)

In [ ]:
gen_datapoints = function(){
  v1 = rnorm(30,3,1)
  v2 = rnorm(30,14,2)
  correlat = cor.test(v1,v2)$p.value
  if (correlat<0.05){
    print('try again')
    return(gen_datapoints())
  }else{
    list(
        v1 = v1,
        v2 = v2
    )
  }
}
data_list = gen_datapoints()
x = data_list$v1
y = 5+x*3+rnorm(30,0,2)
neg_y = 5-x*3+rnorm(30,0,2)
z = data_list$v2
datapoints = data.frame(x=x,y=y,neg_y = neg_y,z=z)
g = datapoints %>% ggplot(aes(x=x,y=y))+geom_point()+geom_hline(yintercept=mean(datapoints$y))+geom_vline(xintercept=mean(datapoints$x))+geom_smooth(method = 'glm')+labs(title = cor(datapoints$x,datapoints$y))
p = ggplotly(g)
p

In [ ]:
g_neg = datapoints %>% ggplot(aes(x=x,y=neg_y))+geom_point()+geom_hline(yintercept=mean(datapoints$neg_y))+geom_vline(xintercept=mean(datapoints$x))+geom_smooth(method = 'glm')+labs(title = cor(datapoints$x,datapoints$neg_y))
p_neg = ggplotly(g_neg)
p_neg

In [ ]:
g2 = datapoints %>% ggplot(aes(x=x,y=z))+geom_point()+geom_hline(yintercept=mean(datapoints$z))+geom_vline(xintercept=mean(datapoints$x))+geom_smooth(method = 'glm')+labs(title = cor(datapoints$x,datapoints$z))
p2 = ggplotly(g2)
p2

从上面的图中可以看到，当两个变量（图1的x和y）相关性高的时候，自变量增大的时候，因变量倾向于同步增大，反之亦然；以x平均值和y平均值划分出的四个象限里，点大多落在1和3象限（正相关），或者2和4象限。
两个变量（图2的x和z）相关性低的时候，自变量增大，因变量不会倾向于同步增大。以x平均值和y平均值划分出的四个象限里，点落在各个象限

另外，我们也用`geom_smooth`绘制了拟合直线。

## 线性回归中的方差分解

对于变量$Y$与常数$\beta_{0}$，令$\sum{(y_i-\beta_{0})^2}$取极小值的$\beta_{0}=\bar{Y}=\frac{\sum{y_i}}{n}$ （根据平均值的基本性质）

In [ ]:
beta_0_seq = seq(10,20,by=0.01)
sse_calc = function(beta_0){
    sse = sum((y-beta_0)^2)
    return(sse)
}

df = data.frame(x=beta_0_seq) %>% rowwise() %>% mutate(sse=sse_calc(x))
g_mean = df %>% ggplot(aes(x=x,y=sse))+geom_point()+geom_vline(xintercept=mean(y))+labs(title=(mean(y)))
p_mean = ggplotly(g_mean)
p_mean

上面的情况是求$\sum{(y_i-\beta_{0})^2}$减去常数的极小值，如果在一个二维的图像上绘制，相当于下图中，选择一条最合适的横线使各点到横线的垂直距离最短

In [ ]:
x_void = seq(1,30)
beta_0 = mean(y)
df_x_void = data.frame(x=x_void,y=y,y_end=beta_0)
g_void = df_x_void %>% ggplot(aes(x=x,y=y))+geom_point()+geom_segment(aes(xend=x,yend=y_end),color='green')+geom_hline(yintercept=beta_0)+geom_text(aes(10,beta_0,label='y=beta0'))
p_void = ggplotly(g_void)
p_void

那么是否可以通过旋转这个横线使得绿色线段的长度平方总和更小呢？

下图中红色线段是各点到均值的垂直距离，绿色线段是各点到拟合直线的垂直距离，蓝色线段表示直线上各交点与均值横线垂直距离

In [ ]:
fit = linearMod <- lm(y ~ x)
df_x_rel = data.frame(x=x,y=y,y_end=fit$coefficients[1]+x*fit$coefficients['x'])
g_rel = df_x_rel %>% ggplot(aes(x=x,y=y,xend=x,yend=y_end))+geom_point()+geom_segment(color='green')+geom_segment(aes(x=x+0.03,xend=x+0.03,yend=mean(y)),color='red')+geom_hline(yintercept=mean(y))+stat_function(fun=function(x){return(fit$coefficients[1]+x*fit$coefficients['x'])})+geom_text(aes(5,3+5*3,label='y=beta0+beta1*x'))
g_rel2 = df_x_rel %>% ggplot(aes(x=x,y=y,xend=x,yend=y_end))+geom_point()+geom_segment(color='green')+geom_segment(aes(x=x+0.03,y=y_end,xend=x+0.03,yend=mean(y)),color='blue')+geom_hline(yintercept=mean(y))+stat_function(fun=function(x){return(fit$coefficients[1]+x*fit$coefficients['x'])})+geom_text(aes(5,3+5*3,label='y=beta0+beta1*x'))
p_rel = ggplotly(g_rel)
p_rel
p_rel2 = ggplotly(g_rel2)
p_rel2

我们可以尝试计算一下下面三个距离之间的关系：

1. 各数据点与均值横线垂直距离的平方的和 $\sum{(y-\bar{y})^2}$
2. 各数据点与拟合直线垂直距离的平方的和 $\sum{(y-\hat{y})^2}$
3. 拟合直线上各交点与均值横线垂直距离的平方和 $\sum{(\hat{y}-\bar{y})^2}$

In [ ]:
#拟合直线为y = 5+3*x
#1. 各数据点与均值横线垂直距离的平方的和 $\sum{(y-\bar{y})^2}$
sse = sum((df_x_rel$y-mean(df_x_rel$y))^2)
#2. 各数据点与拟合直线垂直距离的平方的和 $\sum{(y-\hat{y})^2}$
ssw = sum((df_x_rel$y-df_x_rel$y_end)^2)
#3. 拟合直线上各交点与均值和弦垂直距离的平方和 $\sum{(\hat{y}-\bar{y})^2}$
ssb = sum((df_x_rel$y_end - mean(df_x_rel$y))^2)
print(list(sse=sse,ssw=ssw,ssb=ssb,ssw_ssb=ssw+ssb))

上面图中蓝色线段的长度的平方和与红色线段的长度的平方和的比值，我们记为$R^2$，有$R^2=\frac{ssb}{sse}$，且对于简单线性回归，有$R^2=r^2$

In [ ]:
r = cor(df_x_rel$x,df_x_rel$y)
R2 = ssb/sse
print(list(r=r,R2=R2,r2=r^2))

## 方差分析

从线性回归的角度来看一下方差分析中的SSE，SSB和SSW

- 方差分析的SSE就是所有数据点对整体均值的差的平方和

加入分组这个**变量**以后：

- 方差分析中各组的均值就是模型对该组样本因变量的最佳拟合值
- 根据$SSW=\sum{(y-\hat{y})^2}$，$\hat{y}$为各组的均值，因此可得$SSW=\sum{\sum{(y_{ij}-\bar{y_i})^2}}$
- 根据 $SSB=\sum{(\hat{y}-\bar{y})^2}$,可以得出$SSW=\sum{(\bar{y_i}-\bar{y})^2}$

In [ ]:
y_g1 = rnorm(20,2,1.5)
y_g2 = rnorm(25,4,2)
y_g3 = rnorm(22,5,1.8)
grp = c(rep(1,20),rep(2,25),rep(3,22))
y = c(y_g1,y_g2,y_g3)
df = data.frame(grp=grp,y=y)
df_stat = df %>% group_by(grp) %>% summarise(mean=mean(y),sd=sd(y)) %>% select(grp,mean,sd)
df = left_join(df,df_stat,by='grp')
g_void = df %>% ggplot(aes(x='combine',y=y))+geom_point(aes(y=mean(df$y),color='red'))+geom_errorbar(aes(ymin=mean(df$y)-sd(df$y),ymax=mean(df$y)+sd(df$y)))+geom_jitter(width=0.3)
p_void = ggplotly(g_void)
p_void
g_x = df %>% ggplot(aes(x=grp,y=y))+geom_point(aes(y=df$mean,color='red'))+geom_errorbar(aes(ymin=df$mean-df$sd,ymax=df$mean+df$sd))+geom_jitter(width=0.3)
p_x = ggplotly(g_x)
p_x
